# Notebook de Pedro

In [6]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [11]:

class Metrics_Regression:
    # Diccionario a nivel de clase para almacenar todos los resultados de todas las ejecuciones
    results = {}

    def __init__(self, y_true, y_pred, validation_name, eps=1e-12):
        # Aseguramos arrays 1D float para evitar problemas de tipos/Series
        self.y_true = np.asarray(y_true, dtype=float).ravel()
        self.y_pred = np.asarray(y_pred, dtype=float).ravel()
        self.validation_name = validation_name
        self.eps = float(eps)

    # --- Métricas de regresión ---
    def mse(self):
        return mean_squared_error(self.y_true, self.y_pred)

    def mae(self):
        return mean_absolute_error(self.y_true, self.y_pred)

    def rmse(self):
        return np.sqrt(self.mse())

    def r2(self):
        return r2_score(self.y_true, self.y_pred)

    def mape(self):
        # *100 para porcentaje; eps evita divisiones por cero
        return np.mean(np.abs((self.y_true - self.y_pred) / (np.abs(self.y_true) + self.eps))) * 100.0

    def rmspe(self):
        return np.sqrt(np.mean(((self.y_true - self.y_pred) / (np.abs(self.y_true) + self.eps)) ** 2)) * 100.0

    # Para obtener todas las métricas de una vez, imprimirlas y almacenarlas
    def all_metrics(self):
        calculated_metrics = {
            'MSE': self.mse(),
            'MAE': self.mae(),
            'RMSE': self.rmse(),
            'R2': self.r2(),
            'MAPE': self.mape(),
            'RMSPE': self.rmspe()
        }

        # Guardar resultados
        Metrics_Regression.results[self.validation_name] = calculated_metrics

        # Imprimir con formato
        print(f"\n===== Resultados {self.validation_name} =====")
        for k, v in calculated_metrics.items():
            if k in ('MAPE', 'RMSPE'):
                print(f'{k}: {v:.4f}%')
            else:
                print(f'{k}: {v:.6f}')


# Clase `myLinearRegression`

La clase implementa tres variantes del modelo de **regresión lineal**:
**OLS**, **Ridge**, y **AutoRidge**.
Todas comparten la misma estructura básica: estimar los coeficientes $\beta$ que mejor explican $y \approx X\beta + \beta_0$.

---

* **1. OLS (Ordinary Least Squares)**

**Objetivo:**
$$
\min_{\beta_0,\beta}||y - (\beta_0 + X\beta)||_2^2
$$

**Solución cerrada:**
$$
\hat{\theta} =
\begin{bmatrix}\hat{\beta}_0,\hat{\beta}\end{bmatrix}^T
(X^\top X)^{-1} X^\top y
$$

**Características algorítmicas:**

* Calcula la solución exacta mediante la ecuación normal.
* Sin ningún tipo de regularización.
* Puede ser inestable si las variables están muy correlacionadas (matriz (X^\top X) casi singular).

---

* **2. Ridge Regression (Regularización L2)**

**Objetivo:**
$$
\min_{\beta_0,\beta}; ||y - (\beta_0 + X\beta)||_2^2+\alpha ||\beta||_2^2
$$

**Solución analítica:**
$$
\hat{\theta}_{ridge} =
(X^\top X + \alpha D)^{-1} X^\top y
$$
donde $D = \mathrm{diag}(0,1,\dots,1)$ para **no penalizar el intercepto**.

**Características algorítmicas:**

* Introduce un término de penalización proporcional al cuadrado de los coeficientes.
* Atenúa la varianza y mejora la estabilidad numérica.
* El parámetro $\alpha$ controla el grado de encogimiento de los coeficientes.

---

* **3. AutoRidge (Regularización adaptativa y validación interna)**

**Objetivo:** aplicar una **penalización variable por característica** y encontrar de forma **automática** la intensidad óptima de regularización.

**Etapas algorítmicas**

1. **Penalización adaptativa por varianza**
   Cada coeficiente recibe un peso proporcional a la varianza de su variable:
   $$
   \lambda_j = \frac{\mathrm{Var}(X_j)}{\overline{\mathrm{Var}}}
   $$
   De esta forma, variables más dispersas se penalizan más.

2. **Rejilla de factores de regularización**
   Se prueban varios multiplicadores (k) alrededor de 1:
   $$
   k \in \{1-1.5\gamma, 1-\gamma, 1, 1+\gamma, 1+1.5\gamma\}
   $$
   donde $\gamma$ controla la amplitud de la búsqueda.

3. **Evaluación Leave-One-Out (LOO)**
   Para cada $k$, se calcula la matriz:
   $$
   A = X^\top X + kD
   \quad\text{y}\quad
   \hat{y} = X A^{-1} X^\top y
   $$
   Se estima el **error LOO** mediante:
   $$
   e_i^{LOO} = \frac{y_i - \hat{y}_i}{1 - h_{ii}},
   \qquad
   MSE_{LOO}(k) = \frac{1}{n}\sum_i (e_i^{LOO})^2
   $$
   El $k$ que minimiza $MSE_{LOO}$ se selecciona como óptimo.

4. **Reentrenamiento final**
   Se calcula el modelo definitivo con el mejor (k^*):
   $$
   \hat{\theta}_{final} = (X^\top X + k^* D)^{-1} X^\top y
   $$


In [13]:
class myLinearRegression(BaseEstimator, RegressorMixin):
    def __init__(self, method="ols", alpha=1.0, gamma=0.1):
        self.method   = method
        self.alpha    = alpha
        self.gamma    = gamma
        self.coef_ = None
        self.intercept_ = None

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float).reshape(-1, 1)

        if self.method == 'ols':
            self._fit_ols_(X, y)
        elif self.method == 'ridge':
            self._fit_ridge_(X, y)
        elif self.method == 'autoridge':
            self._fit_autoridge_(X, y)
        else:
            raise ValueError(f"Método desconocido: {self.method}")

        return self 

    def predict(self, X):
        X = np.asarray(X, dtype=float)
        return X @ self.coef_ + self.intercept_


    def _fit_ols_(self, X, y):
        X_ = np.hstack([np.ones((X.shape[0], 1)), X])
        XtX = X_.T @ X_
        Xty = X_.T @ y
        beta = np.linalg.inv(XtX) @ Xty
        self.intercept_ = float(beta[0, 0])
        self.coef_ = beta[1:, 0]

    def _fit_ridge_(self, X, y):
        X_ = np.hstack([np.ones((X.shape[0], 1)), X])
        XtX = X_.T @ X_
        Xty = X_.T @ y
        D = np.eye(X_.shape[1]); D[0, 0] = 0.0  # no penalizar intercepto
        beta = np.linalg.inv(XtX + self.alpha * D) @ Xty
        self.intercept_ = float(beta[0, 0])
        self.coef_ = beta[1:, 0]

    def _fit_autoridge_(self, X, y):
        # 1) penalización por varianza (intercepto no penalizado)
        n, p = X.shape
        X_  = np.hstack([np.ones((n, 1)), X])
        XtX = X_.T @ X_
        Xty = X_.T @ y

        var = X.var(axis=0) + 1e-12                 # (p,)
        base_lambdas = var / var.mean()             # media = 1
        D_base = np.diag(np.hstack(([0.0], base_lambdas)))  # (p+1,p+1)

        # 2) rejilla dependiendo de gamma
        k_grid = np.array([1 - 1.5*self.gamma, 1 - self.gamma,1, 1 + self.gamma, 1 + 1.5*self.gamma])

        best_k = None
        best_mse = np.inf

        for k in k_grid:
            A = XtX + k * D_base
            invA = np.linalg.inv(A)
            beta = invA @ Xty
            y_hat = X_ @ beta

            # 3) MSE LOO (PRESS): e_loo = (y - y_hat) / (1 - h_ii)
            H_diag = np.sum((X_ @ invA) * X_, axis=1)  # diag(X invA X^T)
            resid = (y - y_hat).ravel()
            e_loo = resid / (1.0 - H_diag + 1e-12)
            mse_loo = float(np.mean(e_loo**2))

            if mse_loo < best_mse:
                best_mse = mse_loo
                best_k = k
        # refit con el mejor k
        A = XtX + best_k * D_base
        beta = np.linalg.inv(A) @ Xty
        self.intercept_ = float(beta[0, 0])
        self.coef_ = beta[1:, 0]

In [8]:
datos = pd.read_csv('data/data_regresion.csv')

# 10 Holdout 75-25

In [10]:
# Dividimos los datos en 10 conjuntos de entrenamiento y pruebas
X = datos.drop('Popularity', axis=1)
y = datos['Popularity']
# 1. Inicializar listas para almacenar los 10 conjuntos
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []

# 2. Bucle para crear 10 divisiones y escalados diferentes
NUM_SPLITS = 10

for i in range(NUM_SPLITS):
    # a. División: Usamos 'i' como random_state para asegurar 10 divisiones distintas
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
        X, y, 
        test_size=0.25, 
        random_state=i # CLAVE: Usar 'i' para tener 10 divisiones diferentes
    )
    
    # b. Estandarización
    scaler = StandardScaler()
    
    #  fit_transform en Train
    X_train_scaled = scaler.fit_transform(X_train_temp)
    
    # transform en Test (usando los parámetros aprendidos en Train)
    X_test_scaled = scaler.transform(X_test_temp)
    
    # c. Almacenar los resultados en las listas
    X_train_list.append(X_train_scaled)
    X_test_list.append(X_test_scaled)
    y_train_list.append(y_train_temp)
    y_test_list.append(y_test_temp)

In [17]:
for i in range(NUM_SPLITS):
    # a. Obtener los conjuntos de entrenamiento y prueba actuales
    X_train_current = X_train_list[i]
    X_test_current = X_test_list[i]
    y_train_current = y_train_list[i]
    y_test_current = y_test_list[i]
    
    # b. Entrenar el regresor myLinearRegression
    lr = myLinearRegression(method='autoridge')
    lr.fit(X_train_current, y_train_current)
    y_pred_lr = lr.predict(X_test_current)
    

    # d. Metricas para el holdout actual
    metrics_10holdout_lr = Metrics_Regression(y_test_current, y_pred_lr, f'myLinearRegression Holdout {i+1}')
    metrics_10holdout_lr.all_metrics()


# Calcular las métricas promedio para los 10 holdouts de myLinearRegression
# Del diccionario results extraer las métricas y calcular la media para cada una
# MSE
all_mse = [Metrics_Regression.results[f'myLinearRegression Holdout {i+1}']['MSE'] for i in range(NUM_SPLITS)]
mse_mean_10holdout = np.mean(all_mse)
# MAE
all_mae = [Metrics_Regression.results[f'myLinearRegression Holdout {i+1}']['MAE'] for i in range(NUM_SPLITS)]
mae_mean_10holdout = np.mean(all_mae)
# RMSE
all_rmse = [Metrics_Regression.results[f'myLinearRegression Holdout {i+1}']['RMSE'] for i in range(NUM_SPLITS)]
rmse_mean_10holdout = np.mean(all_rmse)
# R2
all_r2 = [Metrics_Regression.results[f'myLinearRegression Holdout {i+1}']['R2'] for i in range(NUM_SPLITS)]
r2_mean_10holdout = np.mean(all_r2)
# MAPE
all_mape = [Metrics_Regression.results[f'myLinearRegression Holdout {i+1}']['MAPE'] for i in range(NUM_SPLITS)]
mape_mean_10holdout = np.mean(all_mape)
# RMSPE
all_rmspe = [Metrics_Regression.results[f'myLinearRegression Holdout {i+1}']['RMSPE'] for i in range(NUM_SPLITS)]
rmspe_mean_10holdout = np.mean(all_rmspe)


# Imprimir las métricas promedio
print('\n===== Métricas promedio myLinearRegression en 10 Holdouts =====')
print(f'MSE medio: {mse_mean_10holdout:.4f}')
print(f'MAE medio: {mae_mean_10holdout:.4f}')
print(f'RMSE medio: {rmse_mean_10holdout:.4f}')
print(f'R2 medio: {r2_mean_10holdout:.4f}')
print(f'MAPE medio: {mape_mean_10holdout:.4f}')
print(f'RMSPE medio: {rmspe_mean_10holdout:.4f}')


# 1. Crear el diccionario de resultados medios
mean_results = {
    'MSE':   mse_mean_10holdout,
    'MAE':   mae_mean_10holdout,
    'RMSE':  rmse_mean_10holdout,
    'R2':    r2_mean_10holdout,
    'MAPE':  mape_mean_10holdout,
    'RMSPE': rmspe_mean_10holdout
}

# 2. Guardar en Metrics_Regression.results con una clave única
Metrics_Regression.results['myLinearRegression_10_Holdouts_MEAN'] = mean_results


===== Resultados myLinearRegression Holdout 1 =====
MSE: 288.519704
MAE: 13.625248
RMSE: 16.985868
R2: 0.059034
MAPE: 74.8760%
RMSPE: 336.3711%

===== Resultados myLinearRegression Holdout 2 =====
MSE: 281.851256
MAE: 13.382222
RMSE: 16.788426
R2: 0.064059
MAPE: 67.7704%
RMSPE: 316.7773%

===== Resultados myLinearRegression Holdout 3 =====
MSE: 288.169869
MAE: 13.625127
RMSE: 16.975567
R2: 0.057240
MAPE: 66.6737%
RMSPE: 295.2974%

===== Resultados myLinearRegression Holdout 4 =====
MSE: 285.000948
MAE: 13.481417
RMSE: 16.881971
R2: 0.049871
MAPE: 65.8968%
RMSPE: 280.1466%

===== Resultados myLinearRegression Holdout 5 =====
MSE: 288.118659
MAE: 13.612079
RMSE: 16.974058
R2: 0.053591
MAPE: 71.0002%
RMSPE: 309.9980%

===== Resultados myLinearRegression Holdout 6 =====
MSE: 288.024679
MAE: 13.535088
RMSE: 16.971290
R2: 0.058894
MAPE: 74.6303%
RMSPE: 323.7021%

===== Resultados myLinearRegression Holdout 7 =====
MSE: 286.002955
MAE: 13.562946
RMSE: 16.911622
R2: 0.053914
MAPE: 65.8628%
RM